In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tensorflow.keras import utils
from keras.optimizers import adam
from keras.models import Sequential, Model
from keras.layers import Input, InputLayer, Dense, Reshape, Flatten


def create_dense_ae():
    # Размерность кодированного представления
    encoding_dim =512
    # Энкодер
    input_img = Input(shape=(28, 28, 1)) # 28, 28, 1 - размерности строк, столбцов, фильтров одной картинки, без батч-размерности
    # Вспомогательный слой решейпинга
    flat_img = Flatten()(input_img)
    # Кодированное полносвязным слоем представление
    #x = Dense(encoding_dim, activation='sigmoid')(flat_img)
    encoded = Dense(encoding_dim, activation='relu')(flat_img)#x
    # Декодер
    # Раскодированное другим полносвязным слоем изображение
    input_encoded = Input(shape=(encoding_dim,))
    #x = Dense(encoding_dim, activation='sigmoid')(input_encoded)
    flat_decoded = Dense(28*28, activation='relu')(input_encoded)#x
    decoded = Reshape((28, 28, 1))(flat_decoded)

    # Модели, в конструктор первым аргументом передаются входные слои, а вторым выходные слои
    # Другие модели можно так же использовать как и слои
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")    
    return encoder, decoder, autoencoder    
    
def main():    
    
    #созд экземпляры слоев, применяем их, объединив в модель
    #load db
    fashion_mnist = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    train_images = train_images.reshape(60000, 28, 28, 1)
    test_images = test_images.reshape(10000, 28, 28, 1)
 
    train_labels = utils.to_categorical(train_labels, 10)
    test_labels = utils.to_categorical(test_labels, 10)
    #SCALE THIS data DURING 0 AND 1
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    
    opt = adam(lr=0.01)
    encoder, decoder, autoencoder = create_dense_ae()
    autoencoder.compile(optimizer=opt, loss='mean_squared_error')
    
    autoencoder.summary()
    start = time()
    autoencoder.fit(train_images, train_images, epochs=10, batch_size=128)
    train_time = time() - start
    print('\nFCNN Autoencoder train_time: ',train_time)
    
    model = Sequential()
    for layer in encoder.layers:
        model.add(layer)
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    start = time()
    history = model.fit(train_images, train_labels, epochs=10, batch_size=128)
    train_time = time() - start
    
    print('\nFCNN Model train_time:', train_time)
    start = time()
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    test_time = time() - start
    print('\nFCNN Test time: ', test_time)
    print('\nFCNN Test loss: ', test_loss)
    print('\nFCNN Test accuracy: ', test_acc)
    
    
if __name__ == '__main__':
    main()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
encoder (Model)              (None, 512)               401920    
_________________________________________________________________
decoder (Model)              (None, 28, 28, 1)         402192    
Total params: 804,112
Trainable params: 804,112
Non-trainable params: 0
_________________________________________________________________



Epoch 1/10





60000/60000 [==============================] - 12s 207us/step - loss: 0.0213
Epoch 2/10
60000/60000 [==============================] - 12s 193us/step - loss: 0.0106
Epoch 3/10
60000/60000 [==============================] - 11s 191us/step - loss: 0.0090
Epoch 4/10
60000/60000 [==============================] - 12s 192us/step - loss: 0.0083
Epoch 5/10
60000/60